In [1]:
from PIL import Image
import pytesseract
from reportlab.pdfgen import canvas
import cv2
import numpy as np
import os
import layoutparser as lp
from table_transformer import TableExtractionPipeline

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Adjust if needed


In [2]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch

# Load model and processor
processor = DetrImageProcessor.from_pretrained("hustvl/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("hustvl/detr-resnet-50")

def detect_tables(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Post-process
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    boxes = results["boxes"].tolist()
    scores = results["scores"].tolist()
    labels = results["labels"].tolist()

    # Filter for "table" label if using table-specific model
    return boxes


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

RuntimeError: Failed to import transformers.models.detr.modeling_detr because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')

def extract_text_from_box(image_path, box):
    image = cv2.imread(image_path)
    x_min, y_min, x_max, y_max = map(int, box)
    cropped = image[y_min:y_max, x_min:x_max]
    result = ocr.ocr(cropped, cls=True)
    return result


c:\Users\k3n\AppData\Local\Programs\Python\Python312\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to C:\Users\k3n/.paddleocr/whl\det\en\en_PP-OCRv3_det_infer\en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:00<00:00, 8176.55it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to C:\Users\k3n/.paddleocr/whl\rec\en\en_PP-OCRv4_rec_infer\en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:00<00:00, 14461.82it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\k3n/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 3243.72it/s]

[2025/04/30 18:28:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\k3n/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\k3n/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec

In [ ]:
class TableCell:
    def __init__(self, text, bbox):
        self.text = text
        self.bbox = bbox
        self.right = None
        self.down = None

# Simplified layout structure
def build_table_structure(cells):
    # Sort cells by (top, left)
    sorted_cells = sorted(cells, key=lambda c: (c.bbox[1], c.bbox[0]))

    # Assign right/down links (basic grid logic with thresholds)
    for i, cell in enumerate(sorted_cells):
        for j in range(i+1, len(sorted_cells)):
            neighbor = sorted_cells[j]
            # Same row (within Y threshold)
            if abs(cell.bbox[1] - neighbor.bbox[1]) < 15:
                cell.right = neighbor
                break
        # Down (next row start)
        for k in range(i+1, len(sorted_cells)):
            down_neighbor = sorted_cells[k]
            if abs(cell.bbox[0] - down_neighbor.bbox[0]) < 15:
                cell.down = down_neighbor
                break
    return sorted_cells[0]  # Return the top-left cell


In [6]:
import cv2
import os
import numpy as np
from PIL import Image
import pytesseract
from reportlab.pdfgen import canvas
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from paddleocr import PaddleOCR

print('Running OCR on image...')

# === Preprocessing ===
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    denoised = cv2.fastNlMeansDenoisingColored(image, None, 5, 10, 7, 21)
    lab = cv2.cvtColor(denoised, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    os.makedirs("output/exp/hyper", exist_ok=True)
    cv2.imwrite("output/exp/hyper/new-prepro-img.jpg", enhanced)
    return image, enhanced

# === Table Detection ===
def detect_tables(image_pil):
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
    inputs = processor(images=image_pil, return_tensors="pt")
    outputs = model(**inputs)
    size = torch.tensor([image_pil.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=size, threshold=0.9)[0]
    return results["boxes"]

# === OCR Inside Table ===
ocr_model = PaddleOCR(use_angle_cls=True, lang='en')

def extract_table_cells(image_path, table_boxes):
    image = cv2.imread(image_path)
    table_data = []
    for box in table_boxes:
        x0, y0, x1, y1 = map(int, box.tolist())
        crop = image[y0:y1, x0:x1]
        result = ocr_model.ocr(crop, cls=True)
        cell_texts = [line[1][0] for line in result[0]] if result else []
        table_data.append({"box": (x0, y0, x1, y1), "text": cell_texts})
    return table_data

# === Overlay OCR Text on PDF ===
def image_to_searchable_pdf(image_path, pdf_path):
    original_img, preprocessed_img = preprocess_image(image_path)
    height, width = original_img.shape[:2]
    temp_image_path = "temp_image.jpg"
    Image.fromarray(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)).save(temp_image_path)
    pil_image = Image.fromarray(preprocessed_img)
    
    # Detect tables and extract cells
    print("Detecting tables...")
    table_boxes = detect_tables(pil_image)
    print("Extracting table cells...")
    table_cells = extract_table_cells(image_path, table_boxes)

    # Use Tesseract for general layout text
    print("Running Tesseract OCR...")
    custom_config = r'--oem 3 --psm 12 -l eng'
    data = pytesseract.image_to_data(pil_image, output_type=pytesseract.Output.DICT, config=custom_config)

    c = canvas.Canvas(pdf_path, pagesize=(width, height))
    c.drawImage(temp_image_path, 0, 0, width=width, height=height)
    c.setFillColorRGB(255, 255, 255, alpha=1)
    c.setFont("Helvetica", 20)

    lines = {}
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 60 and data['text'][i].strip():
            key = (data['block_num'][i], data['par_num'][i], data['line_num'][i])
            if key not in lines:
                lines[key] = []
            lines[key].append(i)

    for key in lines:
        word_indices = lines[key]
        line_text = " ".join([data['text'][i] for i in word_indices])
        x = min([data['left'][i] for i in word_indices])
        avg_top = int(np.mean([data['top'][i] for i in word_indices]))
        avg_height = int(np.mean([data['height'][i] for i in word_indices]))
        y_adjusted = height - avg_top - int(avg_height * 0.8)
        c.drawString(x, y_adjusted, line_text)

    # Overlay table cell OCR (optional)
    c.setFont("Helvetica", 10)
    for cell in table_cells:
        x0, y0, x1, y1 = cell["box"]
        y_adj = height - y1
        text = " | ".join(cell["text"])
        c.drawString(x0, y_adj + 5, text)

    c.save()
    os.remove(temp_image_path)
    print(f"✅ Searchable PDF saved to: {pdf_path}")

if __name__ == "__main__":
    image_to_searchable_pdf("input/cert.jpg", "output/exp/hyper/new-output.pdf")
    print("Searchable PDF created!")


RuntimeError: Failed to import transformers.models.detr.image_processing_detr because of the following error (look up to see its traceback):
No module named 'google.protobuf'

In [ ]:

from PIL import Image
import pytesseract
from reportlab.pdfgen import canvas
import cv2
import numpy as np
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def preprocess_image(image_path):
    image = cv2.imread(image_path)

    # Gentle denoising
    denoised = cv2.fastNlMeansDenoisingColored(image, None, 5, 10, 7, 21)

    # Convert to LAB and enhance contrast
    lab = cv2.cvtColor(denoised, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    os.makedirs("output/exp/hyper", exist_ok=True)
    cv2.imwrite("output/exp/hyper/new-prepro-img.jpg", enhanced)

    return image, enhanced

def detect_table_lines(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 15, -2)

    horizontal = thresh.copy()
    vertical = thresh.copy()

    # Horizontal lines
    h_size = int(horizontal.shape[1] / 30)
    h_structure = cv2.getStructuringElement(cv2.MORPH_RECT, (h_size, 1))
    horizontal = cv2.erode(horizontal, h_structure)
    horizontal = cv2.dilate(horizontal, h_structure)

    # Vertical lines
    v_size = int(vertical.shape[0] / 30)
    v_structure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    vertical = cv2.erode(vertical, v_structure)
    vertical = cv2.dilate(vertical, v_structure)

    # Combine horizontal and vertical
    mask = cv2.add(horizontal, vertical)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    table_boxes = [cv2.boundingRect(cnt) for cnt in contours if cv2.contourArea(cnt) > 1000]
    return table_boxes

def image_to_searchable_pdf(image_path, pdf_path):
    original_img, preprocessed_img = preprocess_image(image_path)
    height, width = original_img.shape[:2]

    temp_image_path = "temp_image.jpg"
    Image.fromarray(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)).save(temp_image_path)

    custom_config = r'--oem 3 --psm 12 -l eng'
    pil_image = Image.fromarray(preprocessed_img)
    data = pytesseract.image_to_data(pil_image, output_type=pytesseract.Output.DICT, config=custom_config)

    c = canvas.Canvas(pdf_path, pagesize=(width, height))
    c.drawImage(temp_image_path, 0, 0, width=width, height=height)
    c.setFillColorRGB(255, 255, 255, alpha=1)
    c.setFont("Helvetica", 20)

    # Group words by line
    lines = {}
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 60 and data['text'][i].strip():
            key = (data['block_num'][i], data['par_num'][i], data['line_num'][i])
            if key not in lines:
                lines[key] = []
            lines[key].append(i)

    for key in lines:
        word_indices = lines[key]
        line_text = " ".join([data['text'][i] for i in word_indices])
        x = min([data['left'][i] for i in word_indices])
        avg_top = int(np.mean([data['top'][i] for i in word_indices]))
        avg_height = int(np.mean([data['height'][i] for i in word_indices]))
        y_adjusted = height - avg_top - int(avg_height * 0.8)
        c.drawString(x, y_adjusted, line_text)

    # Detect table lines and draw them
    table_boxes = detect_table_lines(preprocessed_img)
    for (x, y, w, h) in table_boxes:
        c.setStrokeColorRGB(1, 0, 0)
        c.rect(x, height - y - h, w, h)

    c.save()
    os.remove(temp_image_path)
    print(f"✅ Searchable PDF saved to: {pdf_path}")

if __name__ == "__main__":
    image_to_searchable_pdf("input/cert.jpg", "output/exp/hyper/new-output.pdf")